In [1]:
import carla
import random
import Controller.PIDController as PIDController
import Controller.MPCController as MPCController
import math
import numpy as np
from Utils.synch_mode import CarlaSyncMode
import time
from Utils.utils import *
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.8.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [12]:
from gymnasium import spaces

In [13]:
observation_space = spaces.Box(low=0, high=255, shape=(128, 128, 1), dtype=np.uint8)

In [14]:
observation_space

Box(0, 255, (128, 128, 1), uint8)

In [ ]:
action_space = spaces.Box(low=-1, high=1,shape=(9,),dtype="float32")

In [ ]:
action = action_space.sample()

In [ ]:
action

In [ ]:
y_values_RL =np.array([0,5])
x_values_RL = np.array([-10,10])
yaw_values_RL = np.array([-3.14,3.14])

In [ ]:
x0 = (max(x_values_RL)-min(x_values_RL))*((action[0]+1)/2)+min(x_values_RL)
y0 = (max(y_values_RL)-min(y_values_RL))*((action[1]+1)/2)+min(y_values_RL)
yaw0 = (max(yaw_values_RL)-min(yaw_values_RL))*((action[2]+1)/2)+min(yaw_values_RL)

x1 = (max(x_values_RL)-min(x_values_RL))*((action[3]+1)/2)+min(x_values_RL)
y1 = (max(y_values_RL)-min(y_values_RL))*((action[4]+1)/2)+min(y_values_RL)+y0
yaw1 = (max(yaw_values_RL)-min(yaw_values_RL))*((action[5]+1)/2)+min(yaw_values_RL)

x2 = (max(x_values_RL)-min(x_values_RL))*((action[6]+1)/2)+min(x_values_RL)
y2 = (max(y_values_RL)-min(y_values_RL))*((action[7]+1)/2)+min(y_values_RL)+y1
yaw2 = (max(yaw_values_RL)-min(yaw_values_RL))*((action[8]+1)/2)+min(yaw_values_RL)

In [ ]:
y2

In [ ]:
y1

In [ ]:
(max(y_values_RL)-min(y_values_RL))*((action[7]+1)/2)+min(y_values_RL)

In [ ]:
waypoints = [[x0, y0, road_desired_speed, yaw0], [x1, y1, road_desired_speed, yaw1], [x2, y2, road_desired_speed, yaw2]]

In [ ]:
collision = 0
lane = 0
finish = 1

In [ ]:
done = 1 if collision or lane or finish else 0

In [ ]:
done

In [ ]:
print(x0) 
print(y0)
print(yaw0)

print(x1)
print(y1)
print(yaw1)

print(x2)
print(y2)
print(yaw2)

In [ ]:
x0 = (max(x_values_RL)-min(x_values_RL))*((action[0]+1)/2)+min(x_values_RL)
x0

In [ ]:
y0 = (max(y_values_RL)-min(y_values_RL))*((action[0]+1)/2)+min(y_values_RL)
y0

In [ ]:
y =np.array([0,20])
x = np.array([-10,10])
yaw = np.array([-1,1])

In [ ]:
import numpy as np
import random
x = np.array([0,20])
# goal : range [0, 1]
x1 = 2*(x - min(x)) / ( max(x) - min(x) )-1
print(x1)

In [ ]:
z = random.uniform(-1,1)
z

In [ ]:
(max(x)-min(x))*((z+1)/2)+min(x)


In [ ]:
( max(x) -x ) / (max(x) - min(x))

In [2]:
from Utils.HUD import HUD as HUD

In [3]:
from  Vehicle_Control import VehicleControl
from World import World


In [6]:
import argparse

argparser = argparse.ArgumentParser(
    description='CARLA Manual Control Client')
argparser.add_argument(
    '-v', '--verbose',
    action='store_true',
    dest='debug',
    help='print debug information')
argparser.add_argument(
    '--host',
    metavar='H',
    default='127.0.0.1',
    help='IP of the host server (default: 127.0.0.1)')
argparser.add_argument(
    '-p', '--port',
    metavar='P',
    default=2000,
    type=int,
    help='TCP port to listen to (default: 2000)')
argparser.add_argument(
    '-a', '--autopilot',
    action='store_true',
    help='enable autopilot')
argparser.add_argument(
    '--res',
    metavar='WIDTHxHEIGHT',
    default='1280x720',
    help='window resolution (default: 1280x720)')
argparser.add_argument(
    '--filter',
    metavar='PATTERN',
    default='vehicle.*',
    help='actor filter (default: "vehicle.*")')
argparser.add_argument(
    '--rolename',
    metavar='NAME',
    default='hero',
    help='actor role name (default: "hero")')
argparser.add_argument(
    '--gamma',
    default=2.2,
    type=float,
    help='Gamma correction of the camera (default: 2.2)')
argparser.add_argument(
    '--map',
    metavar='NAME',
    default='Town04',
    help='simulation map (default: "Town04")')
argparser.add_argument(
    '--spawn_x',
    metavar='x',
    default='-16.75', #town04 = -16.75
    help='x position to spawn the agent')
argparser.add_argument(
    '--spawn_y',
    metavar='y',
    default='-223.55', #town04 = -223.55
    help='y position to spawn the agent')
argparser.add_argument(
    '--random_spawn',        
    metavar='RS',
    default='0',
    type=int,
    help='Random spawn agent')
argparser.add_argument(
    '--vehicle_id',
    metavar='NAME',
    # default='vehicle.jeep.wrangler_rubicon',
    default='vehicle.tesla.model3',
    help='vehicle to spawn, available options : vehicle.audi.a2 vehicle.audi.tt vehicle.carlamotors.carlacola vehicle.citroen.c3 vehicle.dodge_charger.police vehicle.jeep.wrangler_rubicon vehicle.yamaha.yzf vehicle.nissan.patrol vehicle.gazelle.omafiets vehicle.bh.crossbike vehicle.ford.mustang vehicle.bmw.isetta vehicle.audi.etron vehicle.harley-davidson.low rider vehicle.mercedes-benz.coupe vehicle.bmw.grandtourer vehicle.toyota.prius vehicle.diamondback.century vehicle.tesla.model3 vehicle.seat.leon vehicle.lincoln.mkz2017 vehicle.kawasaki.ninja vehicle.volkswagen.t2 vehicle.nissan.micra vehicle.chevrolet.impala vehicle.mini.cooperst')
argparser.add_argument(
    '--vehicle_wheelbase',
    metavar='NAME',
    type=float,
    default='2.89',
    help='vehicle wheelbase used for model predict control')
argparser.add_argument(
    '--waypoint_resolution',
    metavar='WR',
    default='3',
    type=float,
    help='waypoint resulution for control')
argparser.add_argument(
    '--waypoint_lookahead_distance',
    metavar='WLD',
    default='6',
    type=float,
    help='waypoint look ahead distance for control')
argparser.add_argument(
    '--desired_speed',
    metavar='SPEED',
    default='10',
    type=float,
    help='desired speed for highway driving')
argparser.add_argument(
    '--control_mode',
    metavar='CONT',
    default='PID',
    help='Controller')
argparser.add_argument(
    '--planning_horizon',
    metavar='HORIZON',
    type=int,
    default='3',
    help='Planning horizon for MPC')
argparser.add_argument(
    '--time_step',
    metavar='DT',
    default='0.4',
    type=float,
    help='Planning time step for MPC')
argparser.add_argument(
    '--FPS',
    metavar='FPS',
    default='10',
    type=int,
    help='Frame per second for simulation')

args = argparser.parse_args()

args.width, args.height = [int(x) for x in args.res.split('x')]

In [ ]:
def game_loop(args): 
    world=None   
    try: 
        client = carla.Client(args.host, args.port)
        client.set_timeout(100.0)
        hud = HUD()
        # carla_world = client.load_world(args.map)
        carla_world = client.get_world()
        world = World(carla_world, hud, args)
        controller = VehicleControl(world)

        clock = pygame.time.Clock()
        while True:
            # clock.tick(5)
            clock.tick_busy_loop(args.FPS)
            # pygame.time.dalay(500)
            if controller.parse_events(client, world, clock):
                return
            # world.tick(clock)

    finally:

            if world is not None:
                world.destroy()        

In [4]:
from stable_baselines3.common.env_checker import check_env



In [7]:
client = carla.Client(args.host, args.port)
client.set_timeout(100.0)

In [ ]:
carla_world = client.load_world(args.map)

In [8]:
carla_world = client.get_world()

In [9]:
hud = HUD()

In [10]:
world = World(client, carla_world, hud, args)

10.0


In [11]:
# It will check your custom environment and output additional warnings if needed
check_env(world)

vehicle spawned
Control: PID
372850
14760.477743078023


AssertionError: `reset()` must return a tuple (obs, info)

In [ ]:
controller = VehicleControl(world)

In [ ]:
try:   
    world.generate_episode(controller)

finally:

    if world is not None:
        world.destroy()   

In [ ]:
dist = self.world.time_step * current_speed + 0.1
prev_waypoint = self.world.map.get_waypoint(current_location)
current_waypoint = prev_waypoint.next(dist)[0]
# print(current_waypoint)
waypoints = []
road_desired_speed = self.world.desired_speed
# road_desired_speed = world.player.get_speed_limit()/3.6*0.95
for i in range(self.world.planning_horizon):
    if self.world.control_count + i <= 100:
        desired_speed = (self.world.control_count + 1 + i)/100.0 * road_desired_speed
    else:
        desired_speed = road_desired_speed
    dist = self.world.time_step * road_desired_speed
    current_waypoint = prev_waypoint.next(dist)[0]
    waypoints.append([current_waypoint.transform.location.x, current_waypoint.transform.location.y, road_desired_speed, wrap_angle(current_waypoint.transform.rotation.yaw)])
    prev_waypoint = current_waypoint

In [ ]:
try:

    game_loop(args)

except KeyboardInterrupt:
    print('\nCancelled by user. Bye!')

In [ ]:
# Retrieve the spectator object
spectator = world.get_spectator()

# Get the location and rotation of the spectator through its transform
transform = spectator.get_transform()

location = transform.location
rotation = transform.rotation

In [ ]:
## get spectator location.

t = world.get_spectator().get_transform()
# coordinate_str = "(x,y) = ({},{})".format(t.location.x, t.location.y)
coordinate_str = "(x,y,z) = ({},{},{})".format(t.location.x, t.location.y,t.location.z)
print (coordinate_str)